In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-24 22:46:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.32MB/s    in 0.2s    

2024-09-24 22:46:32 (6.32 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open('input.txt', 'r') as f:
    text = f.read()

In [2]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[n] for n in l])

In [5]:
print(encode("hi!! How are you"))
print(decode([46, 47, 2, 2, 1, 20, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]))

[46, 47, 2, 2, 1, 20, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]
hi!! How are you


# Encoding the text

In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

# Training and testing data split

In [7]:
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

# While working with transformers we work with chunks of dataset (All of it would be computationally expensive)

In [8]:
block_size = 8
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
torch.manual_seed(1337)
batch_size = 4
block_szie = 8


def get_batch(split):
    data = train if split == "train" else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
    
xb, yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)



torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both in the shape (B,T), idx turns into B,T,C after getting the logits
        logits = self.token_embedding_table(idx) # takes, (Batch, Time, Channel)
        B, T, C = logits.shape
        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for  _ in range(max_new_tokens):
            logits, loss = self(idx) # Shape is (B, T, C)
            logits = logits[:, -1, :] # Shape is (B, C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # Shape is (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [11]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss)

tensor(5.0364, grad_fn=<NllLossBackward0>)


In [12]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [14]:
batch_size = 32
for steps in range(10000):
    xb , yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss.item())


2.5589075088500977


In [15]:
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


Ong h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd ith henourzincenonthioneir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprroutherccnohathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so itJas
Waketancotha:
h hay.JUCLUKn prids, r loncave w hollular s O:
HIs; ht anjx


# The mathemtical trick used in self-attention

In [16]:
lower_triangular = torch.tril(torch.ones(3,3, dtype=torch.int))
lower_triangular = lower_triangular / torch.sum(lower_triangular, 1, keepdim=True)
b = torch.randint(10, (3,4)).float()
print(lower_triangular)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])


In [17]:
print(b)

tensor([[2., 1., 3., 3.],
        [7., 6., 4., 5.],
        [3., 7., 4., 2.]])


In [18]:
print(lower_triangular @ b)

tensor([[2.0000, 1.0000, 3.0000, 3.0000],
        [4.5000, 3.5000, 3.5000, 4.0000],
        [4.0000, 4.6667, 3.6667, 3.3333]])


In [19]:
B, T, C = 4, 8, 2

In [20]:
x = torch.randn(B, T, C)

In [21]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

In [22]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [23]:
print(xbow2)

tensor([[[ 3.6464e-02, -1.7140e-01],
         [ 3.6715e-01, -1.7754e-01],
         [-8.7498e-02, -4.3437e-02],
         [-3.3560e-01,  1.0641e-01],
         [-2.9223e-01,  3.4569e-01],
         [-2.6991e-01,  2.1853e-01],
         [-2.6965e-01,  7.4588e-02],
         [-2.9767e-01,  4.7604e-01]],

        [[-1.1187e-01,  1.2848e+00],
         [-1.0401e-01,  1.3754e+00],
         [ 5.1072e-01,  6.3233e-01],
         [ 7.6424e-01,  6.1603e-01],
         [ 5.4028e-01,  3.1049e-01],
         [ 2.6961e-01,  1.4276e-01],
         [ 2.3420e-01,  1.1311e-01],
         [-8.2713e-03,  1.3474e-01]],

        [[-1.2228e+00,  1.5673e-01],
         [-6.2128e-01,  2.6980e-01],
         [-2.4584e-01,  9.2436e-01],
         [-1.7305e-01,  5.9401e-01],
         [-4.4866e-01,  5.4448e-01],
         [-3.9372e-01,  7.0349e-01],
         [-3.7888e-01,  6.3211e-01],
         [-3.0721e-01,  4.0055e-01]],

        [[-4.8165e-04, -5.9358e-01],
         [-1.2453e-01, -2.0609e-01],
         [-5.4084e-01, -2.5497e-

# Version3 : Using Softmax (More interesting)

In [24]:

# Wei was being applied in the same way to all batch elements
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=1)

In [25]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

# Self Attention

Key point: 

x is sort of like a private information here. 
For example: say for the 5th token, it has its vector with some information associate about it (position + token emb). Given this information, if you find me interseting, here's what I want to communicate (stored in v)

In [26]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T,T))

wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)

out = wei @ v

# Raw affinities for all the nodes

In [27]:
wei[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0.])

# If I'm a fifth node I would not want to aggrigate any thing above 5th (6, 7, 8 etc). So we need to convert it to the lower triangular form

In [28]:
wei = wei.masked_fill(tril == 0, float("-inf"))
wei[0]

tensor([1., -inf, -inf, -inf, -inf, -inf, -inf, -inf])

# Now we expontiate and normalize

In [29]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Say we're doing sentiment analysis, you would want all nodes to communicate with each other, in that case instead of a lower triangular matrix, you would use the full matrix (In lower triangular information can only flow in the past, not in the future), 

Node:
1) Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregrating information
   with a weighted sum from all nodes that point to them, with data-dependent weights.
2) There is no notion of space. Attention simply acts over a set of vectors. This is why we need to postionally encode tokens.
3) Each example across batch dimention is of course processed completely independently and never "talk" each other.
4) In a "encoder" attention block just delete the single line that does masking with ```tril```, allowing all tokens to communicate

# Self attention: Same source x produces the keys, queries, and value. Every single node with emit 2 vectors, query (what am I loking for?), key (What do I contain), if they're aligned they have a high amount
# Cross Attention: Sometimes the keys come from a separate source 

# Note:

- "Scaled" attention addiitonally divides ```wei``` by $1/{sqrt(head_size)}$. This makes it so which input $Q,K$ are unit variance, wei will be in unit variance too and Softmax will stay diffuse and not saturate too much. Illustration Below:

In [30]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)

wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [31]:
k.var()

tensor(1.0449)

In [32]:
q.var()

tensor(1.0700)

In [33]:
wei.var()

tensor(1.0918)

# Because of softmax (Gives us diffused forms), if wei takes a very positive or very negative numbers inside it, softmax starts to converge the vectors into one-hot. If the wei values are too extreme, softmax will be very peaky